In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
url = 'http://www.ufcstats.com/fight-details/faf15b752fb8924c'
response = requests.get(url)
html_content = response.text
soup = BeautifulSoup(response.text, 'html.parser')


In [2]:
sections = soup.find_all('section', class_='b-fight-details__section js-fight-section')
for section in sections:
    if section.find('table', class_='b-fight-details__table js-fight-table'):
        fight_section = section
        break
else:
    fight_section = None

if fight_section:
    print("Fight section with table found.")
    # Continue with your extraction logic here
else:
    print("Fight section with the desired table not found.")


Fight section with table found.


In [3]:
table = fight_section.find('table', class_='b-fight-details__table js-fight-table')


In [4]:
headers = []
if table:
    header_row = table.find('thead').find_all('th')
    headers = [header.get_text(strip=True) for header in header_row]
    headers.insert(0, 'Round')
    print("Headers:", headers)
else:
    print("Table not found within the section.")


Headers: ['Round', 'Fighter', 'KD', 'Sig. str.', 'Sig. str. %', 'Total str.', 'Td %', 'Td %', 'Sub. att', 'Rev.', 'Ctrl']


In [5]:
if table:
    all_rows_data = []
    current_round = ''  # Variable to keep track of the current round

    # Find all the tbody elements to handle multiple sections within the table
    for tbody in table.find_all('tbody'):
        rows = tbody.find_all(['tr', 'th'])  # Include <th> tags in the search to find round headers
        for row in rows:
            # Check if the row is a round header
            if row.name == 'th':
                # Extract the round number from the header
                current_round = row.get_text(strip=True)
                continue  # Skip further processing for header rows
            
            cells = row.find_all('td')
            # Initialize lists to hold data for two fighters in the row, including the current round
            fighter_1_data = [current_round]
            fighter_2_data = [current_round]
            for cell in cells:
                # Extract texts from each <p> tag within the cell
                stats = [p.get_text(strip=True) for p in cell.find_all('p')]
                if len(stats) == 2:
                    fighter_1_data.append(stats[0])
                    fighter_2_data.append(stats[1])
                elif len(stats) == 1:  # Handling the case where there might be only one stat in a cell
                    fighter_1_data.append(stats[0])
                    fighter_2_data.append('')  # Append an empty string if no stat for the second fighter
                else:  # If no <p> tags or an unexpected number, append placeholders or handle accordingly
                    fighter_1_data.append('N/A')
                    fighter_2_data.append('N/A')
            
            # Append the processed data for both fighters to all_rows_data
            all_rows_data.append(fighter_1_data)
            all_rows_data.append(fighter_2_data)
    
    # Print each row's data for verification
    for row_data in all_rows_data:
        print(row_data)
else:
    print("No table found.")

    


['Round 1', 'Raquel Pennington', '0', '11 of 22', '50%', '26 of 39', '0 of 2', '0%', '0', '0', '0:16']
['Round 1', 'Mayra Bueno Silva', '0', '18 of 22', '81%', '27 of 31', '2 of 2', '100%', '0', '0', '3:20']
['Round 2', 'Raquel Pennington', '0', '33 of 52', '63%', '47 of 66', '0 of 0', '---', '0', '1', '1:04']
['Round 2', 'Mayra Bueno Silva', '0', '20 of 30', '66%', '29 of 41', '0 of 1', '0%', '1', '0', '2:46']
['Round 3', 'Raquel Pennington', '0', '27 of 43', '62%', '52 of 68', '1 of 1', '100%', '0', '0', '2:32']
['Round 3', 'Mayra Bueno Silva', '0', '17 of 20', '85%', '20 of 23', '0 of 2', '0%', '1', '0', '1:35']
['Round 4', 'Raquel Pennington', '0', '29 of 38', '76%', '75 of 86', '0 of 0', '---', '0', '1', '3:17']
['Round 4', 'Mayra Bueno Silva', '0', '7 of 8', '87%', '8 of 10', '1 of 2', '50%', '1', '0', '1:11']
['Round 5', 'Raquel Pennington', '0', '34 of 46', '73%', '65 of 83', '0 of 0', '---', '2', '0', '4:19']
['Round 5', 'Mayra Bueno Silva', '0', '7 of 8', '87%', '12 of 13', '

In [6]:
if table:
    for tbody in table.find_all('tbody'):
        rows = tbody.find_all('tr')
        for row in rows:
            # Assuming the first column contains fighter names with links
            fighter_links = row.find_all('td')[0].find_all('a')
            for link in fighter_links:
                print(f"Fighter Name: {link.text}, URL: {link['href']}")


Fighter Name: Raquel Pennington , URL: http://www.ufcstats.com/fighter-details/fc169c387b4b465d
Fighter Name: Mayra Bueno Silva , URL: http://www.ufcstats.com/fighter-details/cc8c623cca88f54f
Fighter Name: Raquel Pennington , URL: http://www.ufcstats.com/fighter-details/fc169c387b4b465d
Fighter Name: Mayra Bueno Silva , URL: http://www.ufcstats.com/fighter-details/cc8c623cca88f54f
Fighter Name: Raquel Pennington , URL: http://www.ufcstats.com/fighter-details/fc169c387b4b465d
Fighter Name: Mayra Bueno Silva , URL: http://www.ufcstats.com/fighter-details/cc8c623cca88f54f
Fighter Name: Raquel Pennington , URL: http://www.ufcstats.com/fighter-details/fc169c387b4b465d
Fighter Name: Mayra Bueno Silva , URL: http://www.ufcstats.com/fighter-details/cc8c623cca88f54f
Fighter Name: Raquel Pennington , URL: http://www.ufcstats.com/fighter-details/fc169c387b4b465d
Fighter Name: Mayra Bueno Silva , URL: http://www.ufcstats.com/fighter-details/cc8c623cca88f54f


In [7]:

df = pd.DataFrame(all_rows_data, columns=headers)
df

# all_rows_data

,Round,Fighter,KD,Sig. str.,Sig. str. %,Total str.,Td %,Td %,Sub. att,Rev.,Ctrl
0,Round 1,Raquel Pennington,0,11 of 22,50%,26 of 39,0 of 2,0%,0,0,0:16
1,Round 1,Mayra Bueno Silva,0,18 of 22,81%,27 of 31,2 of 2,100%,0,0,3:20
2,Round 2,Raquel Pennington,0,33 of 52,63%,47 of 66,0 of 0,---,0,1,1:04
3,Round 2,Mayra Bueno Silva,0,20 of 30,66%,29 of 41,0 of 1,0%,1,0,2:46
4,Round 3,Raquel Pennington,0,27 of 43,62%,52 of 68,1 of 1,100%,0,0,2:32
5,Round 3,Mayra Bueno Silva,0,17 of 20,85%,20 of 23,0 of 2,0%,1,0,1:35
6,Round 4,Raquel Pennington,0,29 of 38,76%,75 of 86,0 of 0,---,0,1,3:17
7,Round 4,Mayra Bueno Silva,0,7 of 8,87%,8 of 10,1 of 2,50%,1,0,1:11
8,Round 5,Raquel Pennington,0,34 of 46,73%,65 of 83,0 of 0,---,2,0,4:19
9,Round 5,Mayra Bueno Silva,0,7 of 8,87%,12 of 13,0 of 2,0%,0,0,0:06


In [8]:
next_fight_section = fight_section.find_next('section', class_='b-fight-details__section js-fight-section')
if next_fight_section:
    print("Next fight section with table found.")
    next_table = next_fight_section.find('table', class_='b-fight-details__table js-fight-table')
#     if next_table:
#         headers = ['Round']  # Initialize headers with 'Round' as the first element
#         header_row = next_table.find('thead').find_all('th')
#         headers += [header.get_text(strip=True) for header in header_row]
#         print("Headers from the next table:", headers)
        
#         # You can continue with your extraction logic for the next table here
        
#     else:
#         print("Table not found within the next fight section.")
# else:
#     print("Next fight section with the desired table not found.")

Next fight section with table found.


In [9]:
next_fight_section

<section class="b-fight-details__section js-fight-section">
<p class="b-fight-details__collapse-link_tot" style="margin-bottom: 0px;margin-top: 0px;">
        Significant Strikes

      </p>
</section>

In [10]:
# Find the section that contains the "Significant Strikes" marker
significant_strikes_section = soup.find('p', class_='b-fight-details__collapse-link_tot', string=lambda text: 'Significant Strikes' in text if text else False).parent

if significant_strikes_section:
    print("Significant Strikes section found.")
    # Now, find the table within this section
    significant_strikes_table = significant_strikes_section.find_next('table', class_='b-fight-details__table js-fight-table')
    
    if significant_strikes_table:
        headers = []
        # Extract headers from the table, including 'Round' as a manual addition
        # headers = ['Round']  # Initialize headers with 'Round'
        header_row = significant_strikes_table.find('thead').find_all('th')
        headers += [header.get_text(strip=True) for header in header_row]
        print("Headers from the Significant Strikes table:", headers)
        
        # Initialize a list to hold all row data
        all_rows_data = []
        
        # Assuming there are multiple tbody elements for rounds, similar to the first table
        for tbody in significant_strikes_table.find_all('tbody'):
            rows = tbody.find_all('tr')
            for row in rows:
                cells = row.find_all('td')
                row_data = []  # Placeholder for round number or additional logic to insert round info
                for cell in cells:
                    # Extract text from each cell, assuming simple structure; adjust if multiple <p> per cell
                    cell_texts = cell.get_text(separator=' ', strip=True)
                    row_data.append(cell_texts)
                all_rows_data.append(row_data)
        
        # Example of converting to DataFrame if needed
        import pandas as pd
        df_sig_strikes = pd.DataFrame(all_rows_data, columns=headers)
        print(df_sig_strikes.head())  # Display the first few rows of the DataFrame
        
    else:
        print("Table not found within the Significant Strikes section.")
else:
    print("Significant Strikes section not found.")


Significant Strikes section found.
Headers from the Significant Strikes table: ['Fighter', 'Sig. str', 'Sig. str. %', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground']
                               Fighter           Sig. str Sig. str. %  \
0  Raquel Pennington Mayra Bueno Silva  11 of 22 18 of 22     50% 81%   
1  Raquel Pennington Mayra Bueno Silva  33 of 52 20 of 30     63% 66%   
2  Raquel Pennington Mayra Bueno Silva  27 of 43 17 of 20     62% 85%   
3  Raquel Pennington Mayra Bueno Silva    29 of 38 7 of 8     76% 87%   
4  Raquel Pennington Mayra Bueno Silva    34 of 46 7 of 8     73% 87%   

               Head              Body            Leg          Distance  \
0  7 of 18 11 of 13     4 of 4 4 of 5  0 of 0 3 of 4   9 of 19 7 of 11   
1  29 of 47 7 of 14   4 of 5 10 of 11  0 of 0 3 of 5  23 of 40 7 of 16   
2   13 of 28 6 of 8  12 of 13 9 of 10  2 of 2 2 of 2    9 of 23 4 of 6   
3   22 of 31 3 of 3     7 of 7 3 of 4  0 of 0 1 of 1    5 of 11 4 of 5   
4   20 of 32 5 of 

In [11]:
# sig_strikes_p_tag = soup.find('p', class_='b-fight-details__collapse-link_tot', string=lambda text: 'Significant Strikes' in text if text else False)
# sig_strikes_p_tag


df = pd.DataFrame(all_rows_data, columns=headers)
df

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Raquel Pennington Mayra Bueno Silva,11 of 22 18 of 22,50% 81%,7 of 18 11 of 13,4 of 4 4 of 5,0 of 0 3 of 4,9 of 19 7 of 11,2 of 3 10 of 10,0 of 0 1 of 1
1,Raquel Pennington Mayra Bueno Silva,33 of 52 20 of 30,63% 66%,29 of 47 7 of 14,4 of 5 10 of 11,0 of 0 3 of 5,23 of 40 7 of 16,10 of 12 12 of 13,0 of 0 1 of 1
2,Raquel Pennington Mayra Bueno Silva,27 of 43 17 of 20,62% 85%,13 of 28 6 of 8,12 of 13 9 of 10,2 of 2 2 of 2,9 of 23 4 of 6,14 of 16 13 of 14,4 of 4 0 of 0
3,Raquel Pennington Mayra Bueno Silva,29 of 38 7 of 8,76% 87%,22 of 31 3 of 3,7 of 7 3 of 4,0 of 0 1 of 1,5 of 11 4 of 5,5 of 5 3 of 3,19 of 22 0 of 0
4,Raquel Pennington Mayra Bueno Silva,34 of 46 7 of 8,73% 87%,20 of 32 5 of 6,5 of 5 0 of 0,9 of 9 2 of 2,8 of 17 6 of 7,7 of 7 0 of 0,19 of 22 1 of 1


In [26]:
import pandas as pd

# Assuming significant_strikes_table is already found
if significant_strikes_table:
    # Adjust headers to exclude 'Fighter'
    headers = ['Round']  # Starting with 'Round' only
    header_row = significant_strikes_table.find('thead').find_all('th')
    headers += [header.get_text(strip=True) for header in header_row]  # Append other headers

    all_rows_data = []
    current_round = 1  # Initialize round counter

    tbody_elements = significant_strikes_table.find_all('tbody')
    for tbody in tbody_elements:
        rows = tbody.find_all('tr')
        for row in rows:
            # Initialize data rows without 'Fighter' placeholder
            fighter_1_data = [current_round]  # Assume first entry is round number
            fighter_2_data = [current_round]  # Assume first entry is round number

            cells = row.find_all('td')
            for cell in cells:
                p_tags = cell.find_all('p')
                if len(p_tags) == 2:  # Assuming there are always 2 <p> tags for 2 fighters
                    fighter_1_data.append(p_tags[0].get_text(strip=True))
                    fighter_2_data.append(p_tags[1].get_text(strip=True))
                elif len(p_tags) == 1:  # Handling cases with only one <p> tag
                    fighter_1_data.append(p_tags[0].get_text(strip=True))
                    fighter_2_data.append('')  # Placeholder or logic to handle missing data

            # Append each fighter's data as a separate row
            all_rows_data.append(fighter_1_data)
            all_rows_data.append(fighter_2_data)

        # Assume each tbody represents a new round; increment the round counter
        current_round += 1

    # Create DataFrame
    df_sig_strikes = pd.DataFrame(all_rows_data, columns=headers)
    print(df_sig_strikes.head())
else:
    print("Significant Strikes table not found.")


   Round            Fighter  Sig. str Sig. str. %      Head      Body     Leg  \
0      1  Raquel Pennington  11 of 22         50%   7 of 18    4 of 4  0 of 0   
1      1  Mayra Bueno Silva  18 of 22         81%  11 of 13    4 of 5  3 of 4   
2      1  Raquel Pennington  33 of 52         63%  29 of 47    4 of 5  0 of 0   
3      1  Mayra Bueno Silva  20 of 30         66%   7 of 14  10 of 11  3 of 5   
4      1  Raquel Pennington  27 of 43         62%  13 of 28  12 of 13  2 of 2   

   Distance    Clinch  Ground  
0   9 of 19    2 of 3  0 of 0  
1   7 of 11  10 of 10  1 of 1  
2  23 of 40  10 of 12  0 of 0  
3   7 of 16  12 of 13  1 of 1  
4   9 of 23  14 of 16  4 of 4  


In [27]:
df_sig_strikes

,Round,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,1,Raquel Pennington,11 of 22,50%,7 of 18,4 of 4,0 of 0,9 of 19,2 of 3,0 of 0
1,1,Mayra Bueno Silva,18 of 22,81%,11 of 13,4 of 5,3 of 4,7 of 11,10 of 10,1 of 1
2,1,Raquel Pennington,33 of 52,63%,29 of 47,4 of 5,0 of 0,23 of 40,10 of 12,0 of 0
3,1,Mayra Bueno Silva,20 of 30,66%,7 of 14,10 of 11,3 of 5,7 of 16,12 of 13,1 of 1
4,1,Raquel Pennington,27 of 43,62%,13 of 28,12 of 13,2 of 2,9 of 23,14 of 16,4 of 4
5,1,Mayra Bueno Silva,17 of 20,85%,6 of 8,9 of 10,2 of 2,4 of 6,13 of 14,0 of 0
6,1,Raquel Pennington,29 of 38,76%,22 of 31,7 of 7,0 of 0,5 of 11,5 of 5,19 of 22
7,1,Mayra Bueno Silva,7 of 8,87%,3 of 3,3 of 4,1 of 1,4 of 5,3 of 3,0 of 0
8,1,Raquel Pennington,34 of 46,73%,20 of 32,5 of 5,9 of 9,8 of 17,7 of 7,19 of 22
9,1,Mayra Bueno Silva,7 of 8,87%,5 of 6,0 of 0,2 of 2,6 of 7,0 of 0,1 of 1


In [14]:
# sig_strikes_table = soup.find('table', class_='b-fight-details__table js-fight-table')
# # sig_strikes_table

In [15]:
import pandas as pd

# Assuming the headers are correctly defined as per the table structure
headers = ['Round', 'Fighter', 'Sig. str', 'Sig. str. %', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground']

# Initialize the data list
all_fighter_data = []

# Assuming tbody_tags contains each round's data correctly
round_number = 1  # Initialize or dynamically extract round number if available
for tbody in tbody_tags:  # Loop through each tbody assuming each represents a round
    rows = tbody.find_all('tr')
    
    for row in rows:
        # For each fighter (assuming 2 <p> tags per cell for 2 fighters)
        fighter_1_data = [round_number]  # Start each row with round number
        fighter_2_data = [round_number]
        
        cells = row.find_all('td')
        for cell_index, cell in enumerate(cells):
            p_tags = cell.find_all('p')
            if len(p_tags) == 2:  # Assuming there are exactly 2 <p> tags per cell
                # Append data for each fighter to their respective list
                fighter_1_data.append(p_tags[0].get_text(strip=True))
                fighter_2_data.append(p_tags[1].get_text(strip=True))
            elif len(p_tags) == 1:  # Handle case with only one <p> tag if necessary
                # This part might need adjustment based on your specific case
                fighter_1_data.append(p_tags[0].get_text(strip=True))
                fighter_2_data.append('')  # Assuming missing data or similar handling
                
        # After processing all cells, append each fighter's data as a separate row
        all_fighter_data.append(fighter_1_data)
        all_fighter_data.append(fighter_2_data)
        
    round_number += 1  # Increment round number after processing each tbody

# Create DataFrame from the collected data
df = pd.DataFrame(all_fighter_data, columns=headers)
print(df.head())  # Display the first few rows to verify


NameError: name 'tbody_tags' is not defined

In [17]:
df

,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,Raquel Pennington Mayra Bueno Silva,11 of 22 18 of 22,50% 81%,7 of 18 11 of 13,4 of 4 4 of 5,0 of 0 3 of 4,9 of 19 7 of 11,2 of 3 10 of 10,0 of 0 1 of 1
1,Raquel Pennington Mayra Bueno Silva,33 of 52 20 of 30,63% 66%,29 of 47 7 of 14,4 of 5 10 of 11,0 of 0 3 of 5,23 of 40 7 of 16,10 of 12 12 of 13,0 of 0 1 of 1
2,Raquel Pennington Mayra Bueno Silva,27 of 43 17 of 20,62% 85%,13 of 28 6 of 8,12 of 13 9 of 10,2 of 2 2 of 2,9 of 23 4 of 6,14 of 16 13 of 14,4 of 4 0 of 0
3,Raquel Pennington Mayra Bueno Silva,29 of 38 7 of 8,76% 87%,22 of 31 3 of 3,7 of 7 3 of 4,0 of 0 1 of 1,5 of 11 4 of 5,5 of 5 3 of 3,19 of 22 0 of 0
4,Raquel Pennington Mayra Bueno Silva,34 of 46 7 of 8,73% 87%,20 of 32 5 of 6,5 of 5 0 of 0,9 of 9 2 of 2,8 of 17 6 of 7,7 of 7 0 of 0,19 of 22 1 of 1


In [22]:
import pandas as pd

# Define the DataFrame headers based on the expected statistics
headers = ['Round', 'Fighter', 'Sig. str', 'Sig. str. %', 'Head', 'Body', 'Leg', 'Distance', 'Clinch', 'Ground']

# Initialize a list to collect data for DataFrame creation
all_fighter_data = []

# Manually track the round
current_round = 1

# Assuming you have a way to iterate through each set of fighter statistics
for index, stats in enumerate(data_extraction_logic):
    # Check if the index is even or odd to adjust the round number
    if index % 4 == 0 and index != 0:  # Assuming every 4 rows (2 for each fighter) represent a new round
        current_round += 1
    
    fighter_stats = [current_round]  # Start with the round number
    
    # Assuming 'stats' contains the extracted statistics for both fighters, properly split them
    # This example assumes 'stats' is a list containing all required statistics in order
    fighter_stats.extend(stats)  # Add the stats to the current fighter's data list
    
    # Append the completed fighter statistics to the all_fighter_data list
    all_fighter_data.append(fighter_stats)

# Create the DataFrame from the collected data
df = pd.DataFrame(all_fighter_data, columns=headers)

# Display the DataFrame to verify the structure
print(df)


NameError: name 'data_extraction_logic' is not defined

In [21]:
# df = pd.DataFrame(all_fighter_data, columns=headers)
df


,Round,Fighter,Sig. str,Sig. str. %,Head,Body,Leg,Distance,Clinch,Ground
0,1,Raquel Pennington,Mayra Bueno Silva,11 of 22,18 of 22,50%,81%,7 of 18,11 of 13,None
1,4 of 4,4 of 5,0 of 0,3 of 4,9 of 19,7 of 11,2 of 3,10 of 10,0 of 0,1 of 1
2,1,Raquel Pennington,Mayra Bueno Silva,33 of 52,20 of 30,63%,66%,29 of 47,7 of 14,None
3,4 of 5,10 of 11,0 of 0,3 of 5,23 of 40,7 of 16,10 of 12,12 of 13,0 of 0,1 of 1
4,2,Raquel Pennington,Mayra Bueno Silva,27 of 43,17 of 20,62%,85%,13 of 28,6 of 8,None
5,12 of 13,9 of 10,2 of 2,2 of 2,9 of 23,4 of 6,14 of 16,13 of 14,4 of 4,0 of 0
6,2,Raquel Pennington,Mayra Bueno Silva,29 of 38,7 of 8,76%,87%,22 of 31,3 of 3,None
7,7 of 7,3 of 4,0 of 0,1 of 1,5 of 11,4 of 5,5 of 5,3 of 3,19 of 22,0 of 0
8,3,Raquel Pennington,Mayra Bueno Silva,34 of 46,7 of 8,73%,87%,20 of 32,5 of 6,None
9,5 of 5,0 of 0,9 of 9,2 of 2,8 of 17,6 of 7,7 of 7,0 of 0,19 of 22,1 of 1
